In [13]:
import numpy as np
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import classification_report
from collections import Counter
import pandas as pd
from sklearn.model_selection import StratifiedKFold,  cross_val_score
from sklearn.metrics import make_scorer, fbeta_score

In [14]:
# Lade Datensatz
train_data_loaded = pd.read_csv('../data/train_data_2024-08-01.csv')
X = train_data_loaded.drop(columns=['UKATEGORIE'])
y = train_data_loaded['UKATEGORIE']


In [15]:
# StratifiedKFold initialisieren
skf = StratifiedKFold(n_splits=5)

# Variablen für Ergebnisse
fbetas_rf = []
fbetas_random = []
fbetas_mehrheit = []

In [16]:
beta = 2
fbeta_scorer= make_scorer(fbeta_score, beta=beta)

In [17]:
model_logreg = LogisticRegression(class_weight={0: 1, 1: 9}, random_state=42)

logreg_scores = cross_val_score(model_logreg, X, y, cv=skf, scoring=fbeta_scorer)
print(f'LogReg F betas: ', logreg_scores)
print(f'LogReg F-beta Score (mean): {logreg_scores.mean()}')
print(f'LogReg F-beta Score (std): {logreg_scores.std()}')

counter = 1
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Drucke die ersten 10 values von y_train
    print("y_train:", y_train.values[:10])
    # Drucke die ersten 10 values von y_test
    print("y_test:", y_test.values[:10])
    
    # Random Forest Modell trainieren
    model_logreg.fit(X_train, y_train)
    y_pred_rf = model_logreg.predict(X_test)
    print("Ersten 10 Vorhersagen mit dem Modell:", y_pred_rf[:10])
    
    # Zufällige Vorhersagen basierend auf den Klassenwahrscheinlichkeiten erstellen
    class_counts = Counter(y_test)
    total_samples = len(y_test)
    class_probabilities = {cls: count / total_samples for cls, count in class_counts.items()}
    
    np.random.seed(42)
    y_pred_random_weighted = np.random.choice(
        list(class_probabilities.keys()),
        size=y_test.shape,
        p=list(class_probabilities.values())
    )
    
    # Drucke die ersten 10 Werte
    print("Ersten 10 Vorhersagen mit dem _random_weighted:", y_pred_random_weighted[:10])
    
    # Leistung der gewichteten zufälligen Vorhersagen bewerten
    fbeta_random_weighted = fbeta_score(y_test, y_pred_random_weighted, beta=beta, average='weighted')
    print(f"Fold Nummer {counter}: fbeta_random_weighted - Score ist {fbeta_random_weighted}")
    fbetas_random.append(fbeta_random_weighted)
    
    # Immer Mehrheitsklasse vorhersagen
    y_pred_majority = np.full(y_test.shape, y_train.mode().iloc[0])
    
    # Drucke die ersten 10 Werte
    print("Ersten 10 Vorhersagen mit dem y_pred_majority:", y_pred_majority[:10])
    
    # Leistung der Vorhersagen der Mehrheitsklasse bewerten
    fbeta_mehrheit = fbeta_score(y_test, y_pred_majority, beta=beta, average='weighted')
    print(f"Fold Nummer {counter}: fbeta_mehrheit - Score ist {fbeta_mehrheit}")
    fbetas_mehrheit.append(fbeta_mehrheit)
    
    counter += 1
    
    

/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

LogReg F betas:  [0.47542689 0.47829126 0.47612836 0.48153262 0.47763102]
LogReg F-beta Score (mean): 0.47780202800343163
LogReg F-beta Score (std): 0.0021273629133720267
y_train: [1 1 1 1 1 1 1 1 1 1]
y_test: [0 0 0 0 0 0 0 0 0 0]


/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Ersten 10 Vorhersagen mit dem Modell: [1 1 1 1 1 0 1 1 1 1]
Ersten 10 Vorhersagen mit dem _random_weighted: [0 1 0 0 0 0 0 1 0 0]
Fold Nummer 1: fbeta_random_weighted - Score ist 0.7463015685247019
Ersten 10 Vorhersagen mit dem y_pred_majority: [0 0 0 0 0 0 0 0 0 0]
Fold Nummer 1: fbeta_mehrheit - Score ist 0.8168620906598427
y_train: [0 0 0 0 0 0 0 0 0 0]
y_test: [1 1 1 1 1 1 1 1 1 1]


/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Ersten 10 Vorhersagen mit dem Modell: [1 1 1 0 1 1 1 1 1 1]
Ersten 10 Vorhersagen mit dem _random_weighted: [0 0 0 0 0 0 1 0 0 0]
Fold Nummer 3: fbeta_random_weighted - Score ist 0.7309138770091183
Ersten 10 Vorhersagen mit dem y_pred_majority: [0 0 0 0 0 0 0 0 0 0]
Fold Nummer 3: fbeta_mehrheit - Score ist 0.8167143150750469
y_train: [0 0 0 0 0 0 0 0 0 0]
y_test: [1 1 1 1 1 1 1 1 1 1]


/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Ersten 10 Vorhersagen mit dem Modell: [1 0 1 0 1 1 1 1 0 1]
Ersten 10 Vorhersagen mit dem _random_weighted: [0 0 0 0 0 0 1 0 0 0]
Fold Nummer 4: fbeta_random_weighted - Score ist 0.7416122431135633
Ersten 10 Vorhersagen mit dem y_pred_majority: [0 0 0 0 0 0 0 0 0 0]
Fold Nummer 4: fbeta_mehrheit - Score ist 0.8167143150750469
y_train: [0 0 0 0 0 0 0 0 0 0]
y_test: [1 1 1 1 1 1 1 1 1 1]
Ersten 10 Vorhersagen mit dem Modell: [0 1 1 1 1 1 1 1 1 1]
Ersten 10 Vorhersagen mit dem _random_weighted: [0 0 0 0 0 0 1 0 0 0]
Fold Nummer 5: fbeta_random_weighted - Score ist 0.7340082584457402
Ersten 10 Vorhersagen mit dem y_pred_majority: [0 0 0 0 0 0 0 0 0 0]
Fold Nummer 5: fbeta_mehrheit - Score ist 0.8168394026197696


/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [21]:
# Durchschnittliche Genauigkeiten berechnen
mean_scores = np.mean(logreg_scores)
mean_scores_random = np.mean(fbetas_random)
mean_scores_mehrheit = np.mean(fbetas_mehrheit)

In [22]:
print(f"LogReg Mean score: {mean_scores}")
print(f"Weighted Random Prediction Mean score: {mean_scores_random}")
print(f"Weighted Mehrheit Prediction Mean score: {mean_scores_mehrheit}")


LogReg Mean score: 0.47780202800343163
Weighted Random Prediction Mean score: 0.7382213392739826
Weighted Mehrheit Prediction Mean score: 0.8167984428179098


In [23]:

# Klassifikationsberichte für den letzten Fold ausgeben
print("Random Forest Classification Report (Last Fold):")
print(classification_report(y_test, y_pred_rf))

print("Weighted Random Prediction Classification Report (Last Fold):")
print(classification_report(y_test, y_pred_random_weighted))

Random Forest Classification Report (Last Fold):
              precision    recall  f1-score   support

           0       0.91      0.16      0.27      6787
           1       0.16      0.91      0.28      1231

    accuracy                           0.28      8018
   macro avg       0.54      0.54      0.28      8018
weighted avg       0.79      0.28      0.28      8018

Weighted Random Prediction Classification Report (Last Fold):
              precision    recall  f1-score   support

           0       0.84      0.84      0.84      6787
           1       0.14      0.15      0.14      1231

    accuracy                           0.73      8018
   macro avg       0.49      0.49      0.49      8018
weighted avg       0.74      0.73      0.74      8018

